# 1. mysqlbinlog命令介绍
mysqlbinlog是解析mysql的binlog日志的命令。

要想让数据有binlog记录日志功能必须在my.cnf配置文件中开启binlog功能。

### binlog日志，即binary log，是二进制日志文件。
它有两个作用：
- 一是增量备份，即只备份新增的内容；
- 二是用于主从复制等，即主节点维护了一个binlog日志文件，从节点从binlog中同步数据。

我们可以通过binlog日志恢复数据。

下面就介绍一下开启MySQL binlog日志的过程：


1. 登录MySQL后，输入show variables like '%log_bin%'；查看到binlog日志为OFF关闭状态;

2. 开启MySQL binlog日志

退出MySQL，使用vi编辑器修改MySQL的my.cnf配置文件。

server-id表示单个结点的id，这里由于只有一个结点，所以可以把id随机指定为一个数，这里将id设置成52。若集群中有多个结点，则id不能相同。

log_bin = /data/3306/binlog/mysql-bin，是指定binlog日志文件的名字为mysql-bin，以及其存储路径。

log_bin路径的设置：
mkdir /data/3306/binlog
chown -R mysql.mysql /bin/log
chmod 777 /data/3306/binlog

3. 重启MySQL

注意：每次服务器（数据库）重启，服务器会调用flush logs;，新创建一个binlog日志

# 2. mysql的日志文件是什么呢
用来记录mysql内部增删改查等，对mysql数据库所有库的更新内容的记录

# 3 mysqlbinlog参数说明

常用选项：
- -d, --database=name 指定数据库名称，只列出指定的数据库相关操作   
- -o, --offset=number 忽略掉日志中的前number行   
- -r, --result-file=name 将输出的文本格式日志输出到指定文件   
- -s, --short-form 显示简单格式，省略掉一些信息   
- --set-charset=char-name 在输出为文本格式时，在文件第一行加上set names char-name,这个选项在某些情况下装载数据时非常有用   
- --start-datetime=name --stop-datetime=name 指定日期间隔内的所有日志   
- --start-position=# --stop-position=#  指定位置间隔内的所有日志

## 3.1 -d参数，拆分binlog日志，截取指定库的binlog
Mysqlbinlog的-d参数拆分binlog日志，截取指定库的binlog。   
binlog日志里记录了所有库更新的内容的记录，在恢复数据库时不能把所有库更新的内容全部恢复会报错，因为不是所有库都出故障。

-d参数，指定想要恢复的库

> [root@oldboy ~]# mysqlbinlog -d oldboy  /data/3306/mysqlbin_oldboy.000015 >oldboy.sql

## 3.2 --start-position，--stop-position基于位置的增量恢复
指定开始位置和结束位置。
如果只指定开始位置截取从开始位置到结尾。

实践：-r相当于重定向

> [root@localhost data]# mysqlbinlog --no-defaults mysql-bin.000011 --start-position=906 --stop-position=1051 -r pos.sql

## 3.3 --start-datetime,--stop-datetime, 基于时间点的增量恢复（基于时间就不准确了）
指定开始时间和结束时间。

比如我想恢复180201 2:45:16到180201 2:47:01之间的数据。

实践：

> [root@localhost data]# mysqlbinlog --no-defaults mysql-bin.000011 --start-datetime='2018-02-01 2:45:16' --stop-datetime='2018-02-01 2:47:01' -r time.sql

比如说谁谁谁在几点用drop删除了一条记录，那我们在恢复数据的时候就要把这个时间段跳过去。

